In [12]:
%load_ext autoreload 

from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
from easydict import EasyDict as edict
import pandas as pd
import os
from collections import defaultdict
from joblib import Parallel, delayed
import multiprocessing as mp
from IPython.core.debugger import set_trace
from IPython.display import clear_output

from scipy.signal import convolve2d, convolve
from scipy.signal.windows import blackman, gaussian
import copy

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, KernelPCA, FastICA
from sklearn.metrics import r2_score, make_scorer
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.datasets import make_swiss_roll,\
                             make_s_curve,\
                             make_moons

from umap import UMAP

import torch
from torch import nn
from torch import optim
from torch import autograd

from torch.utils.data import DataLoader

from train_utils import get_capacity, plot_weights_hist, train, get_grad_params
from metric_utils import calculate_Q_metrics, \
                         strain, \
                         l2_loss, \
                         to_numpy, \
                         numpy_metric, \
                         cosine_sim

from input_utils import DataGenerator, make_random_affine
from models_utils import MLP_NonlinearEncoder, \
                         init_weights, \
                         universal_approximator, \
                         dJ_criterion, \
                         gained_function, \
                         adjust_learning_rate, \
                         compute_joint_probabilities, \
                         tsne_loss,\
                         tsne_criterion, \
                         sigmoid, \
                         initialize_nonlinearities

from embedding_utils import ConstructUMAPGraph, UMAPLoss, UMAPDataset, umap_criterion_compatibility

from pynndescent import NNDescent
from umap.umap_ import fuzzy_simplicial_set, make_epochs_per_sample
from sklearn.utils import check_random_state

import pygad
from torchga import TorchGA, model2vector, vector2model

import warnings
warnings.filterwarnings("ignore")

plt.rcParams['font.size'] = 20
device = torch.device('cuda:0')
N_CPU = mp.cpu_count()
SEED = 42
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load data 

In [7]:
SCALER = StandardScaler()

input_parameters = {'generator': make_swiss_roll, #make_s_curve, 
                    'generator_kwargs': {'n_samples':1000, 'noise':1e-2}, # 1e-1
                    'unsupervised':True,
                    'whiten':True,
                    'scaler':SCALER,
                    'use_outpt_color':True} 

create_data = DataGenerator(**input_parameters)

inpt, _, color = create_data()

N_TEST = 100

In [8]:
inpt.mean(1), inpt.std(1)

(array([-1.42108547e-17, -3.90798505e-17,  1.59872116e-17]),
 array([1., 1., 1.]))

In [10]:
inpt.max(1), inpt.min(1), (inpt@inpt.T / inpt.shape[1]).round()

(array([1.59902408, 1.67350107, 2.00295631]),
 array([-1.78480188, -1.71298949, -1.66512157]),
 array([[ 1., -0.,  0.],
        [-0.,  1., -0.],
        [ 0., -0.,  1.]]))

In [13]:
inpt_train, inpt_test, color_train, color_test = train_test_split(inpt.T, 
                                                                  color, 
                                                                  random_state=42,
                                                                  test_size=N_TEST)

In [14]:
inpt_train_torch = torch.tensor(inpt_train, dtype=torch.float32).to(device)
inpt_test_torch = torch.tensor(inpt_test, dtype=torch.float32).to(device)

In [15]:
inpt_train_torch.shape, inpt_test_torch.shape

(torch.Size([900, 3]), torch.Size([100, 3]))

# Setup dataloders

In [20]:
graph_constructor = ConstructUMAPGraph(metric='euclidean', 
                                        n_neighbors=15, 
                                        random_state=SEED)

# (epochs_per_sample, head, tail, weight) 
train_graph_data = graph_constructor(inpt_train)
test_graph_data = graph_constructor(inpt_test)

BATCH_SIZE_BP = 2

dataset_train = UMAPDataset(inpt_train, 
                            *train_graph_data, 
                            device=device, 
                            batch_size=BATCH_SIZE_BP)

dataset_test = UMAPDataset(inpt_test, 
                           *test_graph_data, 
                           device=device,
                           batch_size=BATCH_SIZE_BP)


Sun Jul 10 16:36:27 2022 Building RP forest with 7 trees
Sun Jul 10 16:36:27 2022 NN descent for 10 iterations
	 1  /  10
	 2  /  10
	Stopping threshold met -- exiting after 2 iterations
Sun Jul 10 16:36:27 2022 Building RP forest with 5 trees
Sun Jul 10 16:36:27 2022 NN descent for 7 iterations
	 1  /  7
	 2  /  7
	Stopping threshold met -- exiting after 2 iterations


In [24]:
len(dataset_train), len(dataset_test)

(57537, 5565)

In [30]:
criterion_umap = UMAPLoss(device=device, 
                         min_dist=0.1,
                         negative_sample_rate=5,
                         edge_weight=None,
                         repulsion_strength=1.0)

# Adjacent grad

In [80]:
from train_utils import criterion_rule

In [85]:
network = MLP_adjacent().to(device)

In [86]:
opt = optim.Adam(get_grad_params(network.parameters()), lr=1e-3)

In [92]:
# forward pass
maxiter = 1000

losses = []

for itr,batch in tqdm(enumerate(dataset_train)):
    
    if itr >= maxiter:
        break

    assert batch.shape[0] == 2
    X_to, X_from = batch
    X_to, X_from = X_to.unsqueeze(1), X_from.unsqueeze(1)

    
    Y_to = network(X_to)
    Y_from = network(X_from)

    Y = torch.cat([Y_to.T, Y_from.T], dim=0)
    loss_umap = criterion_umap(Y)
    
    opt.zero_grad()
    loss_umap.backward(retain_graph=True, create_graph=True)
    opt.step()

    losses.append(loss_umap.item())


0it [00:00, ?it/s]


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [94]:
# class MLP_adjacent(nn.Module):
    
#     def __init__(self):
    
#         super().__init__()
    
#         self.W_1 = nn.Parameter(torch.zeros(10, 3), requires_grad=False).to(device)
#         self.f_1 = universal_approximator(10, requires_grad=True).to(device)

#         self.W_2 = nn.Parameter(torch.zeros(10, 10), requires_grad=False).to(device)
#         self.f_2 = universal_approximator(10, requires_grad=True).to(device)

#         self.W_3 = nn.Parameter(torch.zeros(2, 10), requires_grad=False).to(device)
#         self.f_3 = universal_approximator(2, requires_grad=True).to(device)
            
#         self.λ = 1e-3    
            
#         init_weights(self)
        
#     def update(self):
        
#         self.W_1 = self.W_1 + self.λ*dW1
#         self.W_1 = self.W_1 + self.λ*dW1
#         self.W_1 = self.W_1 + self.λ*dW1
        

#     def forward(self, X):

#         Y1 = self.f_1(self.W_1@X)
#         dW1 = criterion_rule(X, Y1, self.W_1)
#         self.W_1 = self.W_1 + self.λ*dW1
#         # updated pass
#         Y1 = self.f_1(self.W_1@X)

#         Y2 = self.f_2(self.W_2@Y1)
#         dW2 = criterion_rule(Y1, Y2, self.W_2)
#         self.W_2 = self.W_2 + self.λ*dW2
#         # updated pass
#         Y2 = self.f_2(self.W_2@Y1)

#         Y3 = self.f_3(self.W_3@Y2)
#         dW3 = criterion_rule(Y2, Y3, self.W_3)
#         self.W_3 = self.W_3 + self.λ*dW3
#         # updated pass
#         Y3 = self.f_3(self.W_3@Y2)
        
#         return Y3